In [1]:
# Dependencies
import os
import json

import pandas as pd
import numpy as np
import datetime as dt
import calendar

In [2]:
# Import the ridership CSV file.
df = pd.read_csv(os.path.join('data', '2021', 'bst_ridership.csv'))
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3575182 entries, 0 to 3575181
Data columns (total 9 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   Trip  Duration      3575182 non-null  int64  
 1   Start Station Id    3575182 non-null  int64  
 2   Start Time          3575182 non-null  object 
 3   Start Station Name  3571502 non-null  object 
 4   End Station Id      3573213 non-null  float64
 5   End Time            3575182 non-null  object 
 6   End Station Name    3569288 non-null  object 
 7   Bike Id             3574979 non-null  float64
 8   User Type           3575182 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 245.5+ MB


## Stage 1: Data Type Conversions

In [3]:
df_1 = df.copy()

In [4]:
# Convert `Start Time` and `End Time` to datetime objects for easier downstream temporal manipulation.
df_1['Start Time'] = pd.to_datetime(df_1['Start Time'])
df_1['End Time'] = pd.to_datetime(df_1['End Time'])

In [5]:
# `End Station Id` should be of int64 type like `Start Station Id` but it's of float64 type due to the presence of NaNs when 
# the CSV file was read.
df_1['End Station Id'] = df_1['End Station Id'].astype('Int64')

In [6]:
# Reorder the columns.
df_1 = df_1.iloc[:, [2,5,0,1,3,4,6,7,8]]

In [7]:
df_1.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3575182 entries, 0 to 3575181
Data columns (total 9 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   Start Time          3575182 non-null  datetime64[ns]
 1   End Time            3575182 non-null  datetime64[ns]
 2   Trip  Duration      3575182 non-null  int64         
 3   Start Station Id    3575182 non-null  int64         
 4   Start Station Name  3571502 non-null  object        
 5   End Station Id      3573213 non-null  Int64         
 6   End Station Name    3569288 non-null  object        
 7   Bike Id             3574979 non-null  float64       
 8   User Type           3575182 non-null  object        
dtypes: Int64(1), datetime64[ns](2), float64(1), int64(2), object(3)
memory usage: 248.9+ MB


In [8]:
print('Number of bikes in 2021: ', df_1['Bike Id'].nunique())
print('Number of bike stations in 2021: ', df_1['Start Station Id'].nunique())

Number of bikes in 2021:  6498
Number of bike stations in 2021:  627


## Stage 2: Outliers in Trip Duration
We need to eliminate obvious outliers for `Trip Duration`.

In [9]:
df_2 = df_1.copy()

# Remove the substring `Member` in each record of `User Type`: `Annual Member` to `Annual` and `Casual Member` to `Casual`.
# Replace `Annual` with `Member` for clearer distinction between the two types of riders, causual riders and membership riders.
df_2['User Type'] = df_2['User Type'].str.replace(' Member', '')
df_2['User Type'] = df_2['User Type'].str.replace('Annual', 'Member')

# Rename `Trip Duration` to `Duration`.
df_2.rename(columns={'Trip  Duration': 'Duration'}, inplace=True)

# Limit the trip length to le 1 day but ge 1 min.
print(f"{(df_2['Duration']<60).sum()} trips did not last more than 1 min.\n"
      f"{(df_2['Duration']>(60*60*24)).sum()} trips lasted more than 1 day.")
df_2 = df_2[(df_2['Duration']>=60) & (df_2['Duration']<=(60*60*24))]

11279 trips did not last more than 1 min.
602 trips lasted more than 1 day.


In [10]:
df_2.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3563301 entries, 0 to 3575181
Data columns (total 9 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   Start Time          3563301 non-null  datetime64[ns]
 1   End Time            3563301 non-null  datetime64[ns]
 2   Duration            3563301 non-null  int64         
 3   Start Station Id    3563301 non-null  int64         
 4   Start Station Name  3559633 non-null  object        
 5   End Station Id      3563243 non-null  Int64         
 6   End Station Name    3559329 non-null  object        
 7   Bike Id             3563098 non-null  float64       
 8   User Type           3563301 non-null  object        
dtypes: Int64(1), datetime64[ns](2), float64(1), int64(2), object(3)
memory usage: 275.3+ MB


### `User Type`

In [11]:
# Make a copy of the current DataFrame to prepare for `User Type` summaries.
trips = df_2.copy()

# Insert new columns `Date`, `Month`, `Day`, and `Hour` based on the start time of each recorded trip.
ts = trips['Start Time']
trips = trips.assign(Date=ts.dt.date, 
                     Month=ts.dt.month,
                     Day=ts.dt.weekday,
                     Hour=ts.dt.hour)
trips.rename(columns={'Day': 'Day of Week'}, inplace=True)

# Drop columns that are no longer relevant.
trips.drop(columns=['Start Time', 'End Time', 
                    'Start Station Id', 'Start Station Name',
                    'End Station Id', 'End Station Name', 'Bike Id'], inplace=True)

# Reorder the columns.
trips = trips.iloc[:, [2,3,4,5,0,1]]
trips.head()

,Date,Month,Day of Week,Hour,Duration,User Type
0,2021-01-01,1,4,0,1315,Member
1,2021-01-01,1,4,0,396,Casual
2,2021-01-01,1,4,0,86,Member
3,2021-01-01,1,4,0,741,Casual
4,2021-01-01,1,4,0,2073,Casual


In [12]:
# Create a table that summarizes the trip count by year, month, day of the week, and hour.
# by year
trip_count_by_year = df_2['User Type'].value_counts().sort_index().to_frame(name='Yearly')
# by month
trip_count_by_month = trips.groupby(['Month', 'User Type'])['User Type'].count().unstack().T
trip_count_by_month.columns = trip_count_by_month.columns.map(lambda x:calendar.month_abbr[x])
# by day of the week
trip_count_by_dow = trips.groupby(['Day of Week', 'User Type'])['User Type'].count().unstack().T
trip_count_by_dow.columns = trip_count_by_dow.columns.map(lambda x:calendar.day_abbr[x])
# by hour
trip_count_by_hour = trips.groupby(['Hour', 'User Type'])['User Type'].count().unstack().T
# Concatenate them and export the table to a JSON file.
trip_count = pd.concat([trip_count_by_year, 
                        trip_count_by_month, 
                        trip_count_by_dow,
                        trip_count_by_hour], axis=1)

In [13]:
trip_count

,Yearly,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,...,14,15,16,17,18,19,20,21,22,23
Casual,1428596,24632,5786,34981,73391,183286,154971,170893,189968,166253,...,103147,111864,122094,137995,137440,118094,91326,65449,49070,35672
Member,2134705,60898,52884,121835,150216,239573,296517,291207,305444,287166,...,133243,145617,169258,210173,200118,163526,120886,83938,57466,41340


In [14]:
# Create a table that summarizes the average trip duration by year, month, day of the week, and hour.
# By year
avg_dur_by_year = trips.groupby('User Type')['Duration'].mean().sort_index().to_frame(name='Yearly')
# By month
avg_dur_by_month = trips.groupby(['Month', 'User Type'])['Duration'].mean().unstack().T
avg_dur_by_month.columns = avg_dur_by_month.columns.map(lambda x:calendar.month_abbr[x])
# By day of the week
avg_dur_by_dow = trips.groupby(['Day of Week', 'User Type'])['Duration'].mean().unstack().T
avg_dur_by_dow.columns = avg_dur_by_dow.columns.map(lambda x:calendar.day_abbr[x])
# By hour
avg_dur_by_hour = trips.groupby(['Hour', 'User Type'])['Duration'].mean().unstack().T
# Concatenate them and export the table to a JSON file
avg_dur = pd.concat([avg_dur_by_year, 
                     avg_dur_by_month, 
                     avg_dur_by_dow, 
                     avg_dur_by_hour], axis=1)

In [15]:
avg_dur

,Yearly,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,...,14,15,16,17,18,19,20,21,22,23
User Type,,,,,,,,,,,,,,,,,,,,,
Casual,1351.855550,959.543074,1308.600415,1608.936708,1721.316292,1861.33517,1734.619955,1458.584249,1402.537712,1291.434085,...,1460.158822,1425.608006,1391.598252,1348.790963,1352.098698,1379.756338,1365.364277,1333.486547,1374.461259,1404.427366
Member,766.052013,705.880078,733.691646,766.368293,810.253834,867.99487,826.674143,779.927618,756.049004,724.441748,...,785.948905,785.796919,785.408270,786.694261,793.046957,794.800417,797.871813,785.870631,775.663018,745.274504


In [16]:
# Calculate the total trip duration for each `User Type`.
trips.groupby('User Type')['Duration'].sum().sort_index()

User Type
Casual    1931255432
Member    1635295063
Name: Duration, dtype: int64

## Stage 3: Station Ids

In [17]:
station_info = pd.read_json(os.path.join('data', 'station_info.json'))
stations_todate = station_info.loc[:, ['station_id', 'name', 'lat', 'lon']]
stations_todate.rename(columns={'station_id':'Station Id', 'name':'Name', 'lat':'Lat', 'lon':'Lon'}, inplace=True)
stations_todate.set_index('Station Id', inplace=True)
stations_todate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 655 entries, 7000 to 7772
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Name    655 non-null    object 
 1   Lat     655 non-null    float64
 2   Lon     655 non-null    float64
dtypes: float64(2), object(1)
memory usage: 20.5+ KB


In [18]:
df_3 = df_2.copy()
df_3.drop(columns=['Start Station Name', 'End Station Name', 'Bike Id'], inplace=True)
df_3.dropna(inplace=True) 
df_3.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3563243 entries, 0 to 3575181
Data columns (total 6 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   Start Time        3563243 non-null  datetime64[ns]
 1   End Time          3563243 non-null  datetime64[ns]
 2   Duration          3563243 non-null  int64         
 3   Start Station Id  3563243 non-null  int64         
 4   End Station Id    3563243 non-null  Int64         
 5   User Type         3563243 non-null  object        
dtypes: Int64(1), datetime64[ns](2), int64(2), object(1)
memory usage: 193.7+ MB


In [19]:
# Compute how many stations won't be plotted due to missing coord info.
station_id = df_3['Start Station Id'].unique()
missing_station_id = []
for i in station_id:
    if i not in stations_todate.index:
        missing_station_id.append(i)
len(missing_station_id)

47

In [20]:
# Create a custom aggregation function.
def station_agg(grp):
    features ={
        'Yearly Casual Trips': grp[grp['User Type'] == 'Casual']['Duration'].count(),
        'Yearly Total Trips': grp['Duration'].count(),
        'June Casual Trips': grp[(grp['Start Time'].dt.month == 6) & (grp['User Type'] == 'Casual')]['Duration'].count(),
        'June Casual Total Duration': grp[(grp['Start Time'].dt.month == 6) & (grp['User Type'] == 'Casual')]['Duration'].sum()
    }
    return pd.Series(features)

stations = df_3.groupby('Start Station Id').apply(station_agg).rename_axis('Station Id')

stations.insert(0, 'Yearly Casual Trip Percentage', stations['Yearly Casual Trips']*100/stations['Yearly Total Trips'])
stations['June Casual Trip Average Duration'] = stations['June Casual Total Duration']/stations['June Casual Trips']
stations['June Casual Trip Average Duration'] = stations['June Casual Trip Average Duration'].fillna(0)

stations.drop(columns=['Yearly Casual Trips', 'June Casual Total Duration'], inplace=True)
stations_summary = stations_todate.merge(stations, left_index=True, right_index=True)

In [21]:
# Export JSON files.
trip_count.to_json(os.path.join('data', '2021', 'trip_count_summary.json'), orient='index')
avg_dur.to_json(os.path.join('data', '2021', 'avg_dur_summary.json'), orient='index')
stations_summary.to_json(os.path.join('data', '2021', 'stations_summary.json'), orient='index')